## Set Environment

In [ ]:
# Dependency Env Setting
!pip install efficientnet_pytorch

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import cv2
import PIL
import os
import time
import glob
import pickle
import random
from pathlib import Path
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as T
from torch.utils.data import DataLoader, Dataset, Subset
from efficientnet_pytorch import EfficientNet
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score

# Set random seed
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
os.environ["PYTHONHASHSEED"] = str(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = True
print(f'seed : {seed}')

# Set device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'device : {device}')
print(torch.cuda.get_device_properties(device))

# Set ROOT_PATH
ROOT_PATH = os.getcwd()
print(f'ROOT_PATH : {ROOT_PATH}')

seed : 42
device : cuda:0
_CudaDeviceProperties(name='Tesla P40', major=6, minor=1, total_memory=24451MB, multi_processor_count=30)
ROOT_PATH : /opt/ml


## Define Dataset

In [ ]:
# Define Dataset
class FaceDataset(Dataset) :
    def __init__(self, img_paths, trsf, aug=None, training=False) :
        self.img_paths = img_paths
        self.trsf = trsf
        self.aug = aug
        self.training = training
    
    def __len__(self) :
        return len(self.img_paths)

    def __getitem__(self, idx) :
        img = cv2.imread(self.img_paths[idx])
        
        if self.aug is not None :
            img = self.aug(image=img)['image']
            img = self.trsf(img)
        else : 
            img = self.trsf(img)
        
        if self.training :
            label = labeling(self.img_paths[idx])
            return {'image' : img, 'label' : label}
        else :
            return {'image' : img}

## Define Model 

In [ ]:
# Deffine Model
class MyEfficientNet(nn.Module) :
    def __init__(self) :
        super(MyEfficientNet, self).__init__()
        self.EFF = EfficientNet.from_pretrained('efficientnet-b4', in_channels=3, num_classes=18)
    
    def forward(self, x) :
        x = self.EFF(x)
        x = F.softmax(x, dim=1)
        return x

## Define Transform

In [ ]:
class ToTensor(object) :
    def __call__(self, img) :
        img = np.array(img)/255
        img = img.transpose((2,0,1))
        img = torch.FloatTensor(img)
        return img

In [ ]:
# Set Transform
'각 Model의 Crop Size에 해당하는 Transform을 정의합니다.'
small_trsf = T.Compose([
    T.ToPILImage(),
    T.CenterCrop([300,256]),
    ToTensor(),
    T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
])

midium_trsf = T.Compose([
    T.ToPILImage(),
    T.CenterCrop([384,320]),
    ToTensor(),
    T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
])

big_trsf = T.Compose([
    ToTensor(),
    T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
])

## Inference

In [ ]:
# Ensemble Inference
os.chdir(ROOT_PATH)
submission = pd.read_csv('input/data/eval/info.csv')
test_img_paths = [os.path.join('input/data/eval/images', img_file) for img_file in submission.ImageID]
prediction_lst = []

'최종으로 사용할 Model의 Directory입니다.'
'28. Dropout, B4, NCC'
'27. Dropout, B4, CC'
'26. Dropout, B4'
'25. Seed43, B4, Baseline'

best_dirs=['28. Dropout, B4, NCC', '27. Dropout, B4, CC', '26. Dropout, B4', '25. Seed43, B4, Baseline']
for best_dir in best_dirs :
    if 'NCC' in best_dir  :
        print(best_dir, ' : big')
        dataset = FaceDataset(test_img_paths, big_trsf, training=False)
    elif 'CC' in best_dir :
        print(best_dir, ' : midium')
        dataset = FaceDataset(test_img_paths, midium_trsf, training=False)
    else :
        print(best_dir, ' : small')
        dataset = FaceDataset(test_img_paths, small_trsf, training=False)
    loader = DataLoader(dataset=dataset, batch_size=16, shuffle=False)
    
    best_models = glob.glob(f'custom_data/{best_dir}/*.pth')
    for best_model in best_models :
        model = MyEfficientNet()
        model.load_state_dict(torch.load(best_model))
        model.to(device)
        model.eval()
        prediction_array=[]

        with tqdm(loader,
                 total=loader.__len__(),
                 unit='batch') as test_bar :
            for sample in test_bar :
                imgs = sample['image'].float().to(device)
                probs = model(imgs)
                probs = probs.cpu().detach().numpy()
                prediction_array.extend(probs)

        prediction_lst.append(np.array(prediction_array)[...,np.newaxis])

submission['ans'] = np.argmax(np.mean(np.concatenate(prediction_lst, axis=2), axis=2), axis=1)
submission.to_csv(f'custom_data/{name}/{name}.csv', index=False)

28. Dropout, B4, NCC  : big
Loaded pretrained weights for efficientnet-b4


100%|██████████| 788/788 [04:51<00:00,  2.71batch/s]


Loaded pretrained weights for efficientnet-b4


100%|██████████| 788/788 [04:49<00:00,  2.72batch/s]


Loaded pretrained weights for efficientnet-b4


...

100%|██████████| 788/788 [02:22<00:00,  5.52batch/s]


Loaded pretrained weights for efficientnet-b4


100%|██████████| 788/788 [02:24<00:00,  5.47batch/s]


Loaded pretrained weights for efficientnet-b4


100%|██████████| 788/788 [02:22<00:00,  5.52batch/s]


In [ ]:
Dropout_big_proba = prediction_lst[0:5]
Dropout_midium_proba = prediction_lst[5:10]
Dropout_small_proba = prediction_lst[10:15]
small_proba = prediction_lst[15:20]

In [ ]:
Dropout_Family = []
Dropout_Family.extend(Dropout_big_proba)
Dropout_Family.extend(Dropout_midium_proba)
Dropout_Family.extend(Dropout_small_proba)

if len(Dropout_Family) == 15 :
    submission['ans'] = np.argmax(np.mean(np.concatenate(Dropout_Family, axis=2), axis=2), axis=1)
    submission.to_csv(f'custom_data/Dropout_Family/Dropout_Family.csv', index=False)
else :
    print('Something Wrong')

In [ ]:
Dropout_Big = Dropout_big_proba[:]

if len(Dropout_Big) == 5 :
    submission['ans'] = np.argmax(np.mean(np.concatenate(Dropout_Big, axis=2), axis=2), axis=1)
    submission.to_csv(f'custom_data/Dropout_Big/Dropout_Big.csv', index=False)
else :
    print('Something Wrong')

In [ ]:
Dropout_Midium = Dropout_midium_proba[:]

if len(Dropout_Midium) == 5 : 
    submission['ans'] = np.argmax(np.mean(np.concatenate(Dropout_Midium, axis=2), axis=2), axis=1)
    submission.to_csv(f'custom_data/Dropout_Midium/Dropout_Midium.csv', index=False)
else :
    print('Something Wrong')

In [ ]:
Dropout_Big_Midium = []
Dropout_Big_Midium.extend(Dropout_big_proba)
Dropout_Big_Midium.extend(Dropout_midium_proba)

if len(Dropout_Big_Midium) == 10 :
    submission['ans'] = np.argmax(np.mean(np.concatenate(Dropout_Big_Midium, axis=2), axis=2), axis=1)
    submission.to_csv(f'custom_data/Dropout_Big_Midium/Dropout_Big_Midium.csv', index=False)
else :
    print('Something Wrong')

In [ ]:
Dropout_m_s = []
Dropout_m_s.extend(Dropout_midium_proba)
Dropout_m_s.extend(Dropout_small_proba)

if len(Dropout_m_s) == 10 :
    submission['ans'] = np.argmax(np.mean(np.concatenate(Dropout_m_s, axis=2), axis=2), axis=1)
    submission.to_csv(f'custom_data/Dropout_m_s/Dropout_m_s.csv', index=False)
else :
    print('Something Wrong')

In [ ]:
DM_DS_Nomal = []
DM_DS_Nomal.extend(Dropout_midium_proba)
DM_DS_Nomal.extend(Dropout_small_proba)
DM_DS_Nomal.extend(small_proba)

if len(DM_DS_Nomal) == 15 :
    submission['ans'] = np.argmax(np.mean(np.concatenate(DM_DS_Nomal, axis=2), axis=2), axis=1)
    submission.to_csv(f'custom_data/DM_DS_Nomal/DM_DS_Nomal.csv', index=False)
else :
    print('Something Wrong')